# SU2 BMN model: Lambda=2 - Build H in PauliOps form

In this notebook, the $\Lambda=2$ Hamiltonian at 4 couplings $\lambda = 0.2, 0.5, 1.0, 2.0$ are built from scratch. This notebook must be run with qiskit 0.28 (or older), since the module `opflow' that are crucial for this code is deprecated in later versions. 
https://docs.quantum.ibm.com/api/qiskit/0.41/qiskit.opflow.list_ops.TensoredOp

This notebook is based partly on:
https://github.com/erinaldi/bmn2-qiskit/blob/main/notebooks/QISKIT_susy_matrices_VQE.ipynb

In [1]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse import identity
from scipy.sparse import kron
L=2

In [2]:
import qiskit
from qiskit.opflow import MatrixOp
from qiskit.opflow import ListOp, TensoredOp
from qiskit.opflow import SummedOp
from qiskit.opflow import I

from qiskit.algorithms import NumPyMinimumEigensolver
from scipy.sparse.linalg import eigsh

/opt/anaconda3/lib/python3.9/site-packages/qiskit/circuit/equivalence.py:18: DeprecationWarning: The retworkx package is deprecated and has been renamed to rustworkx. Rustworkx is a drop-in replacement and can be used by replacing `import retworkx` with import `rustworkx`. 
  import retworkx as rx


# Define annihilation operators



In [3]:
X= np.array([[0,1],[1,0]])
Y = np.array([[0, -1j], [1j, 0]])
XY = 0.5*X + 0.5j*Y
X, Y, XY

(array([[0, 1],
        [1, 0]]),
 array([[ 0.+0.j, -0.-1.j],
        [ 0.+1.j,  0.+0.j]]),
 array([[0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j]]))

In [4]:
def to_pauli(op):
    qubitOp = MatrixOp(primitive = op)
    op_pauli = qubitOp.to_pauli_op()
    return op_pauli

In [5]:
a_b = diags(np.sqrt(np.linspace(1,L-1,L-1)),offsets=1)
a_f = diags(np.sqrt(np.linspace(1,1,1)),offsets=1)
i_b = identity(L)
i_f = identity(2)
sz = diags([1.0,-1.0])
a_b.todense(),a_f.todense(), i_b.todense(), i_f.todense(), sz.todense()

(matrix([[0., 1.],
         [0., 0.]]),
 matrix([[0., 1.],
         [0., 0.]]),
 matrix([[1., 0.],
         [0., 1.]]),
 matrix([[1., 0.],
         [0., 1.]]),
 matrix([[ 1.,  0.],
         [ 0., -1.]]))

In [6]:
a_b_pauli = to_pauli(a_b)
i_b_pauli = to_pauli(i_b)
i_f_pauli = to_pauli(i_f)
a_f_pauli = to_pauli(a_f)
sz_p = to_pauli(sz)
a_b_pauli,  a_f_pauli, i_b_pauli, i_f_pauli, sz_p

(SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False),
 SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False),
 PauliOp(Pauli('I'), coeff=1.0),
 PauliOp(Pauli('I'), coeff=1.0),
 PauliOp(Pauli('Z'), coeff=1.0))

# Bosonic space 

In [7]:
N_bos = 6
a_b_list = []  
for i in np.arange(0,N_bos): 
    operator_list = [i_b_pauli] * N_bos
    operator_list[i] = a_b_pauli  
    a_b_list.append(ListOp(operator_list))
    
a_b_list[5]

ListOp([PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False)], coeff=1.0, abelian=False)

In [8]:
a_b_tensor = [TensoredOp(x) for x in a_b_list]
a_b_tensor[2]

TensoredOp([PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0)], coeff=1.0, abelian=False)

# Fermionic space 

In [9]:
N_f = 3 
a_f_list = []  
for i in np.arange(0,N_f):  
    operator_list = [i_f_pauli] * N_f   
    operator_list[i] = a_f_pauli  
    for j in np.arange(0,i): 
        operator_list[j] = sz_p
    a_f_list.append(ListOp(operator_list))  
a_f_list

[ListOp([SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0)], coeff=1.0, abelian=False),
 ListOp([PauliOp(Pauli('Z'), coeff=1.0), SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli('I'), coeff=1.0)], coeff=1.0, abelian=False),
 ListOp([PauliOp(Pauli('Z'), coeff=1.0), PauliOp(Pauli('Z'), coeff=1.0), SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False)], coeff=1.0, abelian=False)]

In [10]:
a_f_tensor = [TensoredOp(x) for x in a_f_list]
a_f_tensor[2]

TensoredOp([PauliOp(Pauli('Z'), coeff=1.0), PauliOp(Pauli('Z'), coeff=1.0), SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False)], coeff=1.0, abelian=False)

# Combine Bosonic and Fermionic spaces

In [11]:
i_b_tot = identity(L**N_bos) 
i_f_tot = identity(2**N_f)
i_b_tot, i_f_tot

(<64x64 sparse matrix of type '<class 'numpy.float64'>'
 	with 64 stored elements (1 diagonals) in DIAgonal format>,
 <8x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 8 stored elements (1 diagonals) in DIAgonal format>)

In [12]:
i_b_tot_pauli = to_pauli(i_b_tot)
i_f_tot_pauli = to_pauli(i_f_tot)
i_b_tot_pauli, i_f_tot_pauli

(PauliOp(Pauli('IIIIII'), coeff=1.0), PauliOp(Pauli('III'), coeff=1.0))

In [13]:
a_b_t2 = []
a_f_t2 = []
for i in range(len(a_b_tensor)):
    a_b_t2.append(ListOp([a_b_tensor[i], i_f_tot_pauli]))
for i in range(len(a_f_tensor)):
    a_f_t2.append(ListOp([i_b_tot_pauli, a_f_tensor[i]]))
    
a_b_tt = [TensoredOp(x) for x in a_b_t2]
a_f_tt = [TensoredOp(x) for x in a_f_t2]

In [14]:
len(a_b_tt), len(a_f_tt), a_b_tt[0], a_b_tt[0].to_pauli_op()

(6,
 3,
 TensoredOp([TensoredOp([SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0)], coeff=1.0, abelian=False), PauliOp(Pauli('III'), coeff=1.0)], coeff=1.0, abelian=False),
 PauliSumOp(SparsePauliOp([[False, False, False, False, False, False, False, False,
                  True, False, False, False, False, False, False, False,
                 False, False],
                [False, False, False, False, False, False, False, False,
                  True, False, False, False, False, False, False, False,
                 False,  True]],
               coeffs=[0.5+0.j , 0. +0.5j]), coeff=1.0))

# Build the Hamiltonian

In [15]:
x_list = []
bosons = a_b_tt
fermions = a_f_tt

# for each boson they are constructed using a and adag
x_list = [1/np.sqrt(2)*(~x + x) for x in bosons]

In [38]:
#[x.to_pauli_op() for x in a_b_tt]

In [16]:
x_list[0], x_list[0].to_pauli_op()

(SummedOp([TensoredOp([TensoredOp([SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=-0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0)], coeff=1.0, abelian=False), PauliOp(Pauli('III'), coeff=1.0)], coeff=1.0, abelian=False), TensoredOp([TensoredOp([SummedOp([PauliOp(Pauli('X'), coeff=0.5), PauliOp(Pauli('Y'), coeff=0.5j)], coeff=1.0, abelian=False), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0), PauliOp(Pauli('I'), coeff=1.0)], coeff=1.0, abelian=False), PauliOp(Pauli('III'), coeff=1.0)], coeff=1.0, abelian=False)], coeff=0.7071067811865475, abelian=False),
 PauliOp(Pauli('XIIIIIIII'), coeff=0.7071067811865475))

In [17]:
[x.to_pauli_op() for x in x_list]

[PauliOp(Pauli('XIIIIIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IXIIIIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IIXIIIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IIIXIIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IIIIXIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IIIIIXIII'), coeff=0.7071067811865475)]

In [18]:
[(~x).to_pauli_op() for x in x_list]

[PauliOp(Pauli('XIIIIIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IXIIIIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IIXIIIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IIIXIIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IIIIXIIII'), coeff=0.7071067811865475),
 PauliOp(Pauli('IIIIIXIII'), coeff=0.7071067811865475)]

## Free H

In [19]:
i_tensor = TensoredOp(ListOp([i_b_pauli] * (N_bos+N_f)))
i_tensor.to_pauli_op()

PauliOp(Pauli('IIIIIIIII'), coeff=1.0)

In [20]:
H_zero=(0.25 * (2 * N_bos - 3 * N_f - 3))*i_tensor
H_list = [H_zero]
for op in a_b_tt:
    H_list.append((~op @ op))
for op in a_f_tt:
    H_list.append((3.0 / 2)*(~op @ op))

In [21]:
H_free = SummedOp(ListOp(H_list))
H_fr_pauli = H_free.to_pauli_op()
len(H_fr_pauli), H_fr_pauli

(10,
 SummedOp([PauliOp(Pauli('IIIIIIIII'), coeff=5.25), PauliOp(Pauli('ZIIIIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIIIIII'), coeff=-0.5), PauliOp(Pauli('IIZIIIIII'), coeff=-0.5), PauliOp(Pauli('IIIZIIIII'), coeff=-0.5), PauliOp(Pauli('IIIIZIIII'), coeff=-0.5), PauliOp(Pauli('IIIIIZIII'), coeff=-0.5), PauliOp(Pauli('IIIIIIZII'), coeff=-0.75), PauliOp(Pauli('IIIIIIIZI'), coeff=-0.75), PauliOp(Pauli('IIIIIIIIZ'), coeff=-0.75)], coeff=1.0, abelian=False))

In [22]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=H_fr_pauli)
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.10f}')

Ground state energy value: 0.0000000000


## With interactions

In [23]:
V_f1a = SummedOp([x_list[0]@(~fermions[1])@(~fermions[2]),\
                x_list[1]@(~fermions[2])@(~fermions[0]),\
                x_list[2] @(~fermions[0])@(~fermions[1])])

V_f1b = SummedOp([x_list[3]@(~fermions[1])@(~fermions[2]),\
               x_list[4]@(~fermions[2])@(~fermions[0]),\
                x_list[5]@(~fermions[0])@(~fermions[1])], coeff= -1j)

V_f2a = SummedOp([x_list[0] @fermions[2] @ fermions[1],\
                  x_list[1]@ fermions[0] @ fermions[2],\
                  x_list[2] @ fermions[1] @ fermions[0]], coeff= -1)

V_f2b = SummedOp([x_list[3] @ fermions[2] @ fermions[1],\
                  x_list[4]@ fermions[0] @ fermions[2],\
                x_list[5] @ fermions[1] @ fermions[0]], coeff= -1j)

V_bf = (2j / np.sqrt(2)) *(V_f1a + V_f1b+ V_f2a + V_f2b)

In [24]:
V_bf.to_pauli_op()

SummedOp([PauliOp(Pauli('IIXIIIYXI'), coeff=0.4999999999999999), PauliOp(Pauli('IIXIIIXYI'), coeff=0.4999999999999999), PauliOp(Pauli('IIIIIXXXI'), coeff=0.4999999999999999), PauliOp(Pauli('IIIIIXYYI'), coeff=-0.4999999999999999), PauliOp(Pauli('IXIIIIYZX'), coeff=-0.4999999999999999), PauliOp(Pauli('IXIIIIXZY'), coeff=-0.4999999999999999), PauliOp(Pauli('IIIIXIXZX'), coeff=-0.4999999999999999), PauliOp(Pauli('IIIIXIYZY'), coeff=0.4999999999999999), PauliOp(Pauli('XIIIIIIYX'), coeff=0.4999999999999999), PauliOp(Pauli('XIIIIIIXY'), coeff=0.4999999999999999), PauliOp(Pauli('IIIXIIIXX'), coeff=0.4999999999999999), PauliOp(Pauli('IIIXIIIYY'), coeff=-0.4999999999999999)], coeff=1.0, abelian=False)

In [25]:
#BOSONS INTERACTION TERM
V_b1 = SummedOp([x_list[2]@x_list[2]@x_list[3]@x_list[3], \
                    x_list[2]@x_list[2]@x_list[4]@x_list[4], \
                    x_list[1]@x_list[1]@x_list[3]@x_list[3], \
                    x_list[1]@x_list[1]@x_list[5]@x_list[5], \
                    x_list[0]@x_list[0]@x_list[4]@x_list[4], \
                    x_list[0]@x_list[0]@x_list[5]@x_list[5]])
V_b2 = SummedOp([x_list[0]@x_list[2]@x_list[3]@x_list[5],\
                    x_list[0]@x_list[1]@x_list[3]@x_list[4],\
                    x_list[1]@x_list[2]@x_list[4]@x_list[5]], coeff = -2.0)
V_b = V_b1 + V_b2

In [26]:
V_b.to_pauli_op()

SummedOp([PauliOp(Pauli('IIIIIIIII'), coeff=1.4999999999999996), PauliOp(Pauli('XXIXXIIII'), coeff=-0.49999999999999983), PauliOp(Pauli('XIXXIXIII'), coeff=-0.49999999999999983), PauliOp(Pauli('IXXIXXIII'), coeff=-0.49999999999999983)], coeff=1.0, abelian=False)

# g2N= 0.2

In [27]:
# t'Hooft coupling
g2N = 0.2
# Number of colors
N = 2
# full hamiltonian
H = H_free + g2N/N*V_b + np.sqrt(g2N/N)*V_bf

In [28]:
Hpauli = H.to_pauli_op()
print(f"Number of terms in H: {len(Hpauli.oplist)}")
Hpauli

Number of terms in H: 25


SummedOp([PauliOp(Pauli('IIIIIIIII'), coeff=5.400000000000002), PauliOp(Pauli('ZIIIIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIIIIII'), coeff=-0.5), PauliOp(Pauli('IIZIIIIII'), coeff=-0.5), PauliOp(Pauli('IIIZIIIII'), coeff=-0.5), PauliOp(Pauli('IIIIZIIII'), coeff=-0.5), PauliOp(Pauli('IIIIIZIII'), coeff=-0.5), PauliOp(Pauli('IIIIIIZII'), coeff=-0.75), PauliOp(Pauli('IIIIIIIZI'), coeff=-0.75), PauliOp(Pauli('IIIIIIIIZ'), coeff=-0.75), PauliOp(Pauli('XXIXXIIII'), coeff=-0.04999999999999999), PauliOp(Pauli('XIXXIXIII'), coeff=-0.04999999999999999), PauliOp(Pauli('IXXIXXIII'), coeff=-0.04999999999999999), PauliOp(Pauli('IIXIIIYXI'), coeff=0.15811388300841894), PauliOp(Pauli('IIXIIIXYI'), coeff=0.15811388300841894), PauliOp(Pauli('IIIIIXXXI'), coeff=0.15811388300841894), PauliOp(Pauli('IIIIIXYYI'), coeff=-0.15811388300841894), PauliOp(Pauli('IXIIIIYZX'), coeff=-0.15811388300841894), PauliOp(Pauli('IXIIIIXZY'), coeff=-0.15811388300841894), PauliOp(Pauli('IIIIXIXZX'), coeff=-0.15811388300841894

In [30]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=Hpauli)
ref_value = result.eigenvalue.real
#0.00328726
print(f'Ground state energy value: {ref_value:.8f}')

Ground state energy value: 0.00328726


In [31]:
with open('L2_BMN_l02_Hpauli.txt','w') as f:
    print(Hpauli,file=f)

# g2N = 0.5

In [32]:
# t'Hooft coupling
g2N = 0.5
# Number of colors
N = 2
# full hamiltonian
H = H_free + g2N/N*V_b + np.sqrt(g2N/N)*V_bf

In [33]:
Hpauli = H.to_pauli_op()
print(f"Number of terms in H: {len(Hpauli.oplist)}")
Hpauli

Number of terms in H: 25


SummedOp([PauliOp(Pauli('IIIIIIIII'), coeff=5.625), PauliOp(Pauli('ZIIIIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIIIIII'), coeff=-0.5), PauliOp(Pauli('IIZIIIIII'), coeff=-0.5), PauliOp(Pauli('IIIZIIIII'), coeff=-0.5), PauliOp(Pauli('IIIIZIIII'), coeff=-0.5), PauliOp(Pauli('IIIIIZIII'), coeff=-0.5), PauliOp(Pauli('IIIIIIZII'), coeff=-0.75), PauliOp(Pauli('IIIIIIIZI'), coeff=-0.75), PauliOp(Pauli('IIIIIIIIZ'), coeff=-0.75), PauliOp(Pauli('XXIXXIIII'), coeff=-0.12499999999999996), PauliOp(Pauli('XIXXIXIII'), coeff=-0.12499999999999996), PauliOp(Pauli('IXXIXXIII'), coeff=-0.12499999999999996), PauliOp(Pauli('IIXIIIYXI'), coeff=0.24999999999999994), PauliOp(Pauli('IIXIIIXYI'), coeff=0.24999999999999994), PauliOp(Pauli('IIIIIXXXI'), coeff=0.24999999999999994), PauliOp(Pauli('IIIIIXYYI'), coeff=-0.24999999999999994), PauliOp(Pauli('IXIIIIYZX'), coeff=-0.24999999999999994), PauliOp(Pauli('IXIIIIXZY'), coeff=-0.24999999999999994), PauliOp(Pauli('IIIIXIXZX'), coeff=-0.24999999999999994), PauliOp(P

In [35]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=Hpauli)
ref_value = result.eigenvalue.real
#0.01690123
print(f'Ground state energy value: {ref_value:.8f}')

Ground state energy value: 0.01690123


In [36]:
with open('L2_BMN_l05_Hpauli.txt','w') as f:
    print(Hpauli,file=f)

# g2N = 1.0

In [37]:
# t'Hooft coupling
g2N = 1.0
# Number of colors
N = 2
# full hamiltonian
H = H_free + g2N/N*V_b + np.sqrt(g2N/N)*V_bf

In [38]:
Hpauli = H.to_pauli_op()
print(f"Number of terms in H: {len(Hpauli.oplist)}")
Hpauli

Number of terms in H: 25


SummedOp([PauliOp(Pauli('IIIIIIIII'), coeff=6.0), PauliOp(Pauli('ZIIIIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIIIIII'), coeff=-0.5), PauliOp(Pauli('IIZIIIIII'), coeff=-0.5), PauliOp(Pauli('IIIZIIIII'), coeff=-0.5), PauliOp(Pauli('IIIIZIIII'), coeff=-0.5), PauliOp(Pauli('IIIIIZIII'), coeff=-0.5), PauliOp(Pauli('IIIIIIZII'), coeff=-0.75), PauliOp(Pauli('IIIIIIIZI'), coeff=-0.75), PauliOp(Pauli('IIIIIIIIZ'), coeff=-0.75), PauliOp(Pauli('XXIXXIIII'), coeff=-0.24999999999999992), PauliOp(Pauli('XIXXIXIII'), coeff=-0.24999999999999992), PauliOp(Pauli('IXXIXXIII'), coeff=-0.24999999999999992), PauliOp(Pauli('IIXIIIYXI'), coeff=0.35355339059327373), PauliOp(Pauli('IIXIIIXYI'), coeff=0.35355339059327373), PauliOp(Pauli('IIIIIXXXI'), coeff=0.35355339059327373), PauliOp(Pauli('IIIIIXYYI'), coeff=-0.35355339059327373), PauliOp(Pauli('IXIIIIYZX'), coeff=-0.35355339059327373), PauliOp(Pauli('IXIIIIXZY'), coeff=-0.35355339059327373), PauliOp(Pauli('IIIIXIXZX'), coeff=-0.35355339059327373), PauliOp(Pau

In [40]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=Hpauli)
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.8f}')

Ground state energy value: 0.04829007


In [41]:
with open('L2_BMN_l10_Hpauli.txt','w') as f:
    print(Hpauli,file=f)

# g2N = 2.0

In [42]:
# t'Hooft coupling
g2N = 2.0
# Number of colors
N = 2
# full hamiltonian
H = H_free + g2N/N*V_b + np.sqrt(g2N/N)*V_bf

In [43]:

Hpauli = H.to_pauli_op()
print(f"Number of terms in H: {len(Hpauli.oplist)}")
Hpauli

Number of terms in H: 25


SummedOp([PauliOp(Pauli('IIIIIIIII'), coeff=6.75), PauliOp(Pauli('ZIIIIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIIIIII'), coeff=-0.5), PauliOp(Pauli('IIZIIIIII'), coeff=-0.5), PauliOp(Pauli('IIIZIIIII'), coeff=-0.5), PauliOp(Pauli('IIIIZIIII'), coeff=-0.5), PauliOp(Pauli('IIIIIZIII'), coeff=-0.5), PauliOp(Pauli('IIIIIIZII'), coeff=-0.75), PauliOp(Pauli('IIIIIIIZI'), coeff=-0.75), PauliOp(Pauli('IIIIIIIIZ'), coeff=-0.75), PauliOp(Pauli('XXIXXIIII'), coeff=-0.49999999999999983), PauliOp(Pauli('XIXXIXIII'), coeff=-0.49999999999999983), PauliOp(Pauli('IXXIXXIII'), coeff=-0.49999999999999983), PauliOp(Pauli('IIXIIIYXI'), coeff=0.4999999999999999), PauliOp(Pauli('IIXIIIXYI'), coeff=0.4999999999999999), PauliOp(Pauli('IIIIIXXXI'), coeff=0.4999999999999999), PauliOp(Pauli('IIIIIXYYI'), coeff=-0.4999999999999999), PauliOp(Pauli('IXIIIIYZX'), coeff=-0.4999999999999999), PauliOp(Pauli('IXIIIIXZY'), coeff=-0.4999999999999999), PauliOp(Pauli('IIIIXIXZX'), coeff=-0.4999999999999999), PauliOp(Pauli('II

In [44]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=Hpauli)
#0.08385
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.8f}')

Ground state energy value: 0.08385032


In [45]:
with open('L2_BMN_l20_Hpauli.txt','w') as f:
    print(Hpauli,file=f)